# Notebook 2 - KPI Analysis & Operational Insights

This notebook analyzes the processed daily airline KPI dataset to evaluate:
- Overall operational reliability
- Monthly performance trends
- Carrier-level differences
- Route & airport disruption concentration
- Seasonal performance patterns

All metrics are volume-weighted to accurately reflect system-wide operational impact.


## 1. Load Processed Daily KPI Dataset

The dataset below was prepared in Notebook 1 and contains daily operational KPIs aggregated at the route-carrier level.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

daily_kpi = pd.read_csv("/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/airline_kpi_daily.csv", parse_dates = ['FL_DATE'])

In [2]:
daily_kpi.shape


(2814071, 14)

In [3]:
daily_kpi.head()

,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,DEST,ROUTE,FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,CANCEL_RATE,DIVERT_RATE,DELAY_RATE,SEVERE_DELAY_RATE,ON_TIME_RATE,AVG_ARR_DELAY_MIN
0,2023-01-01,9E,ABE,ATL,ABE-ATL,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2023-01-01,9E,ABY,ATL,ABY-ATL,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2023-01-01,9E,AEX,ATL,AEX-ATL,2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,249.5
3,2023-01-01,9E,AGS,ATL,AGS-ATL,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2023-01-01,9E,ALB,DTW,ALB-DTW,2,0.0,0.0,0.0,0.0,0.5,0.5,0.5,151.0


## 2. Define Arrived Flights for Weighted Calculations

Delay-based metrics should only reflect flights that were completed.
Arrived flights are calculated as total flights minus cancellations and diversions.

In [4]:
daily_kpi['ARRIVED_FLIGHTS'] = daily_kpi['FLIGHTS'] - daily_kpi['CANCELLED_FLIGHTS'] - daily_kpi['DIVERTED_FLIGHTS']

In [5]:
daily_kpi["ARRIVED_FLIGHTS"].describe()

count    2.814071e+06
mean     2.396316e+00
std      2.056337e+00
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      3.500000e+01
Name: ARRIVED_FLIGHTS, dtype: float64

## 3. Overall KPI Snapshot

This section summarizes system-wide airline reliability.
All rates are weighted by flight volume to reflect overall operational impact.

In [6]:
total_flights = daily_kpi['FLIGHTS'].sum()
total_arrived = daily_kpi['ARRIVED_FLIGHTS'].sum()

In [7]:
overall_cancel_rate = daily_kpi['CANCELLED_FLIGHTS'].sum() / total_flights
overall_divert_rate = daily_kpi['DIVERTED_FLIGHTS'].sum() / total_flights
overall_delay_rate = (daily_kpi['DELAY_RATE'] * daily_kpi['ARRIVED_FLIGHTS']).sum() / total_arrived
overall_severe_delay_rate = (daily_kpi['SEVERE_DELAY_RATE'] * daily_kpi['ARRIVED_FLIGHTS']).sum() / total_arrived
overall_avg_arr_delay = (daily_kpi['AVG_ARR_DELAY_MIN'] * daily_kpi['ARRIVED_FLIGHTS']).sum() / total_arrived

In [8]:
kpi_snapshot = {
    "Total Flights": total_flights,
    "Cancellation Rate": overall_cancel_rate,
    "Diversion Rate": overall_divert_rate,
    "Delay Rate": overall_delay_rate,
    "Severe Delay Rate": overall_severe_delay_rate,
    "Avg Arrival Delay Rate (min)": overall_avg_arr_delay
}

In [9]:
kpi_snapshot

{'Total Flights': 6847899,
 'Cancellation Rate': 0.012842333100999298,
 'Diversion Rate': 0.0024170917240455797,
 'Delay Rate': 0.19866479896503308,
 'Severe Delay Rate': 0.1275263946813805,
 'Avg Arrival Delay Rate (min)': 15.231403346143876}

## 4. Monthly Performance Trends

This section evaluates how delay and cancellation rates change over time to identify seasonal patterns and operational shifts.

In [10]:
daily_kpi['MONTH'] = daily_kpi['FL_DATE'].dt.to_period('M').dt.to_timestamp()

In [11]:
kpi_by_month = (
    daily_kpi
    .groupby('MONTH', as_index=False)
    .agg(
        TOTAL_FLIGHTS=('FLIGHTS', 'sum'),
        TOTAL_CANCELLED=('CANCELLED_FLIGHTS', 'sum'),
        TOTAL_DIVERTED=('DIVERTED_FLIGHTS', 'sum'),
        TOTAL_ARRIVED=('ARRIVED_FLIGHTS', 'sum'),
        
        DELAYED_FLIGHTS_W=('DELAY_RATE', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
        SEV_DELAYED_FLIGHTS_W=('SEVERE_DELAY_RATE', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
        ARR_DELAY_MIN_W=('AVG_ARR_DELAY_MIN', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
    )
)

kpi_by_month['CANCEL_RATE'] = kpi_by_month['TOTAL_CANCELLED'] / kpi_by_month['TOTAL_FLIGHTS']
kpi_by_month['DIVERT_RATE'] = kpi_by_month['TOTAL_DIVERTED'] / kpi_by_month['TOTAL_FLIGHTS']
kpi_by_month['DELAY_RATE'] = kpi_by_month['DELAYED_FLIGHTS_W'] / kpi_by_month['TOTAL_ARRIVED']
kpi_by_month['SEVERE_DELAY_RATE'] = kpi_by_month['SEV_DELAYED_FLIGHTS_W'] / kpi_by_month['TOTAL_ARRIVED']
kpi_by_month['AVG_ARR_DELAY_MIN'] = kpi_by_month['ARR_DELAY_MIN_W'] / kpi_by_month['TOTAL_ARRIVED']


In [12]:
output_path = "/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_month.csv"
kpi_by_month.to_csv(output_path, index=False)
output_path

'/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_month.csv'

In [13]:
fig = px.line(
    kpi_by_month.sort_values("MONTH"),
    x = "MONTH",
    y = "DELAY_RATE",
    markers = True,
    title = "Delay Rate Trends Across the Year"
)
fig.update_yaxes(tickformat=".0%")
fig.show()


In [14]:
fig = px.line(
    kpi_by_month.sort_values("MONTH"),
    x = "MONTH",
    y = "CANCEL_RATE",
    markers = True,
    title = "Cancellation Rate Trends Across the Year"
)
fig.update_yaxes(tickformat = ".0%")
fig.show()

In [15]:
fig = px.line(
    kpi_by_month.sort_values("MONTH"),
    x = "MONTH",
    y = "AVG_ARR_DELAY_MIN",
    markers = True,
    title = "Average Arrival Delay Rate Trends Across the Year"
)
fig.show()

## 5. Carrier-Level Performance Comparison

This section compares operational reliability across airlines to identify performance variation and potential operational differences.

In [16]:
kpi_by_carrier = (
    daily_kpi
    .groupby('OP_UNIQUE_CARRIER', as_index=False)
    .agg(
        TOTAL_FLIGHTS=('FLIGHTS', 'sum'),
        TOTAL_CANCELLED=('CANCELLED_FLIGHTS', 'sum'),
        TOTAL_DIVERTED=('DIVERTED_FLIGHTS', 'sum'),
        TOTAL_ARRIVED=('ARRIVED_FLIGHTS', 'sum'),
        
        DELAYED_FLIGHTS_W=('DELAY_RATE', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
        SEV_DELAYED_FLIGHTS_W=('SEVERE_DELAY_RATE', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
        ARR_DELAY_MIN_W=('AVG_ARR_DELAY_MIN', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
    )
)

kpi_by_carrier['CANCEL_RATE'] = kpi_by_carrier['TOTAL_CANCELLED'] / kpi_by_carrier['TOTAL_FLIGHTS']
kpi_by_carrier['DIVERT_RATE'] = kpi_by_carrier['TOTAL_DIVERTED'] / kpi_by_carrier['TOTAL_FLIGHTS']
kpi_by_carrier['DELAY_RATE'] = kpi_by_carrier['DELAYED_FLIGHTS_W'] / kpi_by_carrier['TOTAL_ARRIVED']
kpi_by_carrier['SEVERE_DELAY_RATE'] = kpi_by_carrier['SEV_DELAYED_FLIGHTS_W'] / kpi_by_carrier['TOTAL_ARRIVED']
kpi_by_carrier['AVG_ARR_DELAY_MIN'] = kpi_by_carrier['ARR_DELAY_MIN_W'] / kpi_by_carrier['TOTAL_ARRIVED']

In [17]:
output_path = "/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_carrier.csv"
kpi_by_carrier.to_csv(output_path, index=False)
output_path

'/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_carrier.csv'

In [18]:
carrier_filtered = kpi_by_carrier[kpi_by_carrier["TOTAL_FLIGHTS"] >= 5000].copy()

In [19]:
fig = px.bar(
    carrier_filtered.sort_values("DELAY_RATE", ascending=False),
    x = "OP_UNIQUE_CARRIER",
    y = "DELAY_RATE",
    hover_data = ["TOTAL_FLIGHTS", "CANCEL_RATE", "AVG_ARR_DELAY_MIN"],
    title = "Delay Rate by Carrier (Volume-Filtered)"
)
fig.update_yaxes(tickformat = ".0%")
fig.show()

In [20]:
fig = px.bar(
    carrier_filtered.sort_values("SEVERE_DELAY_RATE", ascending=False),
    x = "OP_UNIQUE_CARRIER",
    y = "SEVERE_DELAY_RATE",
    hover_data = ["TOTAL_FLIGHTS", "CANCEL_RATE", "AVG_ARR_DELAY_MIN"],
    title = "Severe Delay Rate by Carrier (Volume-Filtered)"
)
fig.update_yaxes(tickformat = ".0%")
fig.show()

In [21]:
fig = px.bar(
    carrier_filtered.sort_values("DELAY_RATE", ascending=False),
    x = "OP_UNIQUE_CARRIER",
    y = "CANCEL_RATE",
    hover_data = ["TOTAL_FLIGHTS", "DELAY_RATE", "AVG_ARR_DELAY_MIN"],
    title = "Cancellation Rate by Carrier (Volume-Filtered)"
)
fig.update_yaxes(tickformat = ".0%")
fig.show()

## 6. High-Risk Route Analysis

This section identifies routes with elevated delay rates after applying a minimum flight-volume threshold.

In [22]:
kpi_by_route = (
    daily_kpi
    .groupby('ROUTE', as_index=False)
    .agg(
        TOTAL_FLIGHTS=('FLIGHTS', 'sum'),
        TOTAL_CANCELLED=('CANCELLED_FLIGHTS', 'sum'),
        TOTAL_ARRIVED=('ARRIVED_FLIGHTS', 'sum'),
        
        DELAYED_FLIGHTS_W=('DELAY_RATE', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
        ARR_DELAY_MIN_W=('AVG_ARR_DELAY_MIN', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
    )
)

kpi_by_route['CANCEL_RATE'] = kpi_by_route['TOTAL_CANCELLED'] / kpi_by_route['TOTAL_FLIGHTS']
kpi_by_route['DELAY_RATE'] = kpi_by_route['DELAYED_FLIGHTS_W'] / kpi_by_route['TOTAL_ARRIVED']
kpi_by_route['AVG_ARR_DELAY_MIN'] = kpi_by_route['ARR_DELAY_MIN_W'] / kpi_by_route['TOTAL_ARRIVED']

In [23]:
output_path = "/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_route.csv"
kpi_by_route.to_csv(output_path, index=False)
output_path

'/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_route.csv'

In [24]:
kpi_by_route_filtered = kpi_by_route[kpi_by_route["TOTAL_FLIGHTS"] >= 300].copy()
worst_routes = kpi_by_route_filtered.sort_values("DELAY_RATE", ascending=False).head(15)

In [25]:
fig = px.bar(
    worst_routes.sort_values("DELAY_RATE"),
    x = "DELAY_RATE",
    y = "ROUTE",
    hover_data = ["TOTAL_FLIGHTS", "CANCEL_RATE", "AVG_ARR_DELAY_MIN"],
    title = "Worst Routes by Delay Rate (Min Flight Volume Applied)"
)
fig.update_yaxes(tickformat = ".0%")
fig.show()

## 7. Origin Airport Disruption Concentration

This section highlights airports with elevated operational risk.

In [26]:
kpi_by_origin = (
    daily_kpi
    .groupby('ORIGIN', as_index=False)
    .agg(
        TOTAL_FLIGHTS=('FLIGHTS', 'sum'),
        TOTAL_CANCELLED=('CANCELLED_FLIGHTS', 'sum'),
        TOTAL_ARRIVED=('ARRIVED_FLIGHTS', 'sum'),
        
        DELAYED_FLIGHTS_W=('DELAY_RATE', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
        ARR_DELAY_MIN_W=('AVG_ARR_DELAY_MIN', lambda s: (s * daily_kpi.loc[s.index, 'ARRIVED_FLIGHTS']).sum()),
    )
)

kpi_by_origin['CANCEL_RATE'] = kpi_by_origin['TOTAL_CANCELLED'] / kpi_by_origin['TOTAL_FLIGHTS']
kpi_by_origin['DELAY_RATE'] = kpi_by_origin['DELAYED_FLIGHTS_W'] / kpi_by_origin['TOTAL_ARRIVED']
kpi_by_origin['AVG_ARR_DELAY_MIN'] = kpi_by_origin['ARR_DELAY_MIN_W'] / kpi_by_origin['TOTAL_ARRIVED']

In [ ]:
output_path = "/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_origin.csv"
kpi_by_origin.to_csv(output_path, index=False)
output_path

'/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/kpi_by_origin.csv'

In [28]:
kpi_by_origin_filtered = kpi_by_origin[kpi_by_origin["TOTAL_FLIGHTS"] >= 300].copy()
worst_origin = kpi_by_origin_filtered.sort_values("DELAY_RATE", ascending=False).head(15)

In [29]:
fig = px.bar(
    worst_origin.sort_values("DELAY_RATE"),
    x = "DELAY_RATE",
    y = "ORIGIN",
    hover_data = ["TOTAL_FLIGHTS", "CANCEL_RATE", "AVG_ARR_DELAY_MIN"],
    title = "Worst Airports by Delay Rate (Min Flight Volume Applied)"
)
fig.update_xaxes(tickformat = ".0%")
fig.show()

## 8. Seasonal Performance Patterns

Monthly comparisons reveal recurring patterns in delay and cancellation performance.

In [30]:
fig = px.bar(
    kpi_by_month.sort_values("MONTH", ascending=False),
    x = "MONTH",
    y = "DELAY_RATE",
    hover_data=['TOTAL_FLIGHTS','AVG_ARR_DELAY_MIN'],
    title="Delay Rate by Month (Seasonality View)"
)
fig.update_yaxes(tickformat=".0%")
fig.show()

In [31]:
fig = px.bar(
    kpi_by_month.sort_values("MONTH", ascending=False),
    x = "MONTH",
    y = "CANCEL_RATE",
    hover_data=['TOTAL_FLIGHTS','AVG_ARR_DELAY_MIN'],
    title="Cancellation Rate by Month (Seasonality View)"
)
fig.update_yaxes(tickformat=".0%")
fig.show()

## 9. Key Findings & Business Implications

- Across approximately 6.8 million flights, roughly 20% arrived late, indicating persistent delay pressure.
- Cancellation rates remain relatively low (~1.3%), though winter and summer months exhibit elevated disruption.
- Carrier-level analysis shows meaningful performance variation.
- Certain routes and origin airports drive disproportionate delay concentration.
- Seasonal trends suggest operational strain during peak demand and adverse weather periods.
- Targeted improvements in high-risk months and hubs could meaningfully improve system-wide reliability.

Key Findings

Write 5–7 bullets in plain language:

 Overall reliability summary

 Seasonal disruption pattern

 Best vs worst carriers

 High-risk routes or airports

 Operational implications

## 1. Overall Operational Reliability

Over the analyzed period, airlines operated **6,847,898** flights.

Approximately **1.3 %** were cancelled and **0.24 %** were diverted, indicating that **overall reliability is stable** and **disruptions remain relatively low.**

Among completed flights, **19.87 %** arrived late, with **12.75 %** experiencing significant delays.

The average arrival delay was **15.23 minutes**, suggesting **manageable** operational delay impact.

---

## 2. Time Trends Across the Year

Monthly trend analysis shows that delay and cancellation performance **fluctuated** over the year.

- The **highest delay rate** occurred in **July**, reaching approximately **28 %**.
- The **lowest delay rate** occurred in **November**, at roughly **13 %.**
- Cancellation rates were **[stable / seasonal / spiking]**, with notable increases in **July**.

These patterns suggest **[weather impact / peak travel congestion / operational recovery / staffing constraints]** as potential drivers of performance changes.

---

## 3. Carrier Performance Differences

Carrier-level comparison reveals meaningful variation in operational reliability.

- **Best-performing carrier(s):** **Republic Airline (YX)**, with delay rates near **13 %**.
- **Worst-performing carrier(s):** **Frontier Airline Inc. (F9)**, with delay rates approaching **31 %**.
- Some carriers show **consistent performance**, indicating **[operational strategy or network structure differences]**.

This suggests that **carrier-specific operational practices** play a significant role in overall system performance.

---

## 4. High-Risk Routes and Airports

Route and airport analysis highlights concentrated areas of disruption.

- The **highest-delay routes** include **SNA-PVU**, with delay rates above **50.48%** despite meaningful flight volume.
- Certain **origin airports**, such as **SCK**, show elevated **delay** rates, indicating localized operational constraints.

These findings imply that **targeted operational improvements at specific hubs or corridors** could meaningfully reduce system-wide delays.

---

## 5. Seasonal Performance Patterns

Seasonality analysis shows that performance is not uniform across the calendar year.

- **Summer months** exhibit the **highest cancellation and delay rates**, likely due to **weather, congestion, and demand surges**.
- **November and December** demonstrate relatively stable operations, suggesting **windows of operational efficiency**.

Understanding these recurring patterns can support **staffing, scheduling, and contingency planning** in future operations.

---

## 6. Operational Implications

Overall, the analysis indicates that:

- Airline disruptions are **primarily delay-driven**.
- Performance varies meaningfully by **time of year, carrier, and airport**.
- **Targeted interventions** in high-risk months, carriers, or hubs could **reduce delays and improve passenger experience**.

These insights provide a foundation for **data-driven operational planning and reliability improvement**.
